In [23]:
import struct
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import math
data = open("statistics_03500_repo.bin", "rb").read()
file=[]
with open('re3500_rsm', "r") as f:
    for line in f:
        file.append([float(n) for n in line.strip().split(',')])
file=np.array(file)

In [33]:
def AnisotropyTensor(x):
    k=(np.trace(x)/2)
    a=np.divide(x,2*k)-((1/3)*np.eye(3))
    alambda,aEpsilon = LA.eig(a)
    II=alambda[0]*alambda[0]+alambda[1]*alambda[1]+alambda[0]*alambda[1]
    III=-alambda[0]*alambda[1]*(alambda[0]+alambda[1])
    epsilon=np.power(np.abs(III/2),1/3)* np.sign(III/2)
    eta=np.sqrt(II/3)
    xb=alambda[0]-alambda[1]+(3/2)*alambda[2]+(1/2)
    yb=(np.sqrt(3)/2)*(3*alambda[2]+1)
    return a,k,alambda,aEpsilon,III,II,epsilon,eta,xb,yb
def EulerAngles(x,string):
    #euler angles "x-convection" based on cartesian local system
    SMALL=1e-16
    #Xl = Z1 x Z
    #normalizing
    x=np.einsum('ij,j->ij',x,1/LA.norm(x,axis=0))
    Xl=np.array([x[1,2],-x[0,2],0])
    #X1’x = X’·X1 = X’x ·X1x + X’y ·X1y + X’z ·X1z = Z1y ·X1x – Z1x ·X1y
    X1lx=np.einsum('i,i->', Xl, x[:,0])
    X1ly=np.einsum('i,i->', Xl, x[:,1])
    Z1xy=np.array([x[0,2],x[1,2],0])
    Z1xy=LA.norm(Z1xy)
    if Z1xy>SMALL:
        alpha=np.arctan2(X1ly, X1lx)
        beta=np.arctan2(Z1xy,x[2,2])
        gamma=-1.*np.arctan2(-x[0,2],x[1,2])
    else:
        alpha=0.
        if np.abs(x[2,2])>SMALL:
            beta=0.
        else:
            beta=np.pi
        gamma=-1.*np.arctan2(X1ly, X1lx)
    if string=='graus':
        return np.array([alpha,beta,gamma])*360/(2*np.pi)
    else:
        return np.array([alpha,beta,gamma])
def EulerInverse(alfa,beta,gamma,base):
    A=np.array([[np.cos(alfa),np.sin(alfa),0],[-np.sin(alfa),np.cos(alfa),0],[0,0,1]])
    B=np.array([[1,0,0],[0,np.cos(beta),np.sin(beta)],[0,-np.sin(beta),np.cos(beta)]])
    C=np.array([[np.cos(gamma),np.sin(gamma),0],[-np.sin(gamma),np.cos(gamma),0],[0,0,1]])
    CB=np.einsum('ij,jk->ik', C, B)
    CBA=np.einsum('ij,jk->ik', CB, A)
    QBase=np.einsum('ij,jk->ik', CBA, base)
    return QBase

In [27]:
#Leitura de parâmetros gerais
(my,mz,ubulk,utau,fnu,tstat)=struct.unpack("iidddd", data[:40])

In [28]:
#Leitura de coordenadas
n=my*8+40
y=struct.unpack("d"*my, data[40:n])
m=n
n+=mz*8
z=struct.unpack("d"*mz, data[m:n])

In [29]:
#Leitura das velocidades médias
m=n
n+=my*mz*8
um=struct.unpack("d"*my*mz, data[m:n])
um=np.reshape(um,(my,mz))
m=n
n+=my*mz*8
vm=struct.unpack("d"*my*mz, data[m:n])
vm=np.reshape(vm,(my,mz))
m=n
n+=my*mz*8
wm=struct.unpack("d"*my*mz, data[m:n])
wm=np.reshape(wm,(my,mz))

In [30]:
#Leitura dos termos do tensor de Reynolds
m=n
n+=my*mz*8
uu=struct.unpack("d"*my*mz, data[m:n])
uu=np.reshape(uu,(my,mz))
m=n
n+=my*mz*8
uv=struct.unpack("d"*my*mz, data[m:n])
uv=np.reshape(uv,(my,mz))
m=n
n+=my*mz*8
uw=struct.unpack("d"*my*mz, data[m:n])
uw=np.reshape(uw,(my,mz))
m=n
n+=my*mz*8
vv=struct.unpack("d"*my*mz, data[m:n])
vv=np.reshape(vv,(my,mz))
m=n
n+=my*mz*8
vw=struct.unpack("d"*my*mz, data[m:n])
vw=np.reshape(vw,(my,mz))
m=n
n+=my*mz*8
ww=struct.unpack("d"*my*mz, data[m:n])
ww=np.reshape(ww,(my,mz))

In [32]:
#Parametros DNS
XDNS=np.meshgrid(y, z)
ReStressDNS=np.array([[uu,uv,uw],[uv,vv,vw],[uw,vw,ww]])
VelMediaDNS=np.array([um,vm,wm])
XDNS=np.transpose(XDNS,(1,2,0))
ReStressDNS=np.transpose(ReStressDNS,(2,3,0,1))
VelMediaDNS=np.transpose(VelMediaDNS,(1,2,0))

In [ ]:
#Parametros RANS
X_RANS=np.array((file[:,1],file[:,2])).transpose()
Vel_RANS=np.array((file[:,30],file[:,31],file[:,32])).transpose()
P_RANS=file[:,17]
GradVel_RANS=np.transpose(np.array([[file[:,7],file[:,8],file[:,9]],
                  [file[:,10],file[:,11],file[:,12]],
                  [file[:,13],file[:,14],file[:,15]]]),(2,0,1))
DivVel_RANS = np.array((file[:,7],file[:,11],file[:,15])).transpose()
GradP_RANS = np.array((file[:,4],file[:,5],file[:,6])).transpose()
ReStress_RANS=np.transpose(np.array([[file[:,18],file[:,19],file[:,20]],
                   [file[:,19],file[:,21],file[:,22]],
                   [file[:,20],file[:,22],file[:,23]]]),(2,0,1))
K_RANS=file[:,26]
Eps_RANS=file[:,24]
TurbVisc_RANS=file[:,16]
GradK_RANS=np.array((file[:,27],file[:,28],file[:,29])).transpose()
Yplus_RANS=file[:,33]
WallDist_RANS=file[:,3]
D_RANS=0.5*(GradVel_RANS+np.transpose(GradVel_RANS,(0,2,1)))
W_RANS=0.5*(GradVel_RANS-np.transpose(GradVel_RANS,(0,2,1)))
DDT_RANS=np.einsum('abc,ace->abe', D_RANS, np.transpose(D_RANS,(0,2,1)))
Dnorm_RANS=np.sqrt(np.trace(DDT_RANS,axis1=1,axis2=2))
WWT_RANS=np.einsum('vux,vxz->vuz', W_RANS, np.transpose(W_RANS,(0,2,1)))
Wnorm_RANS=np.sqrt(np.trace(WWT,axis1=1,axis2=2))
DD_RANS=np.einsum('abc,ace->abe', D_RANS, D_RANS)
WW_RANS=np.einsum('vux,vxz->vuz', W_RANS, W_RANS)
ReStReStT_RANS=np.einsum('vux,vxz->vuz', ReStress_RANS, np.transpose(ReStress_RANS,(0,2,1)))
ReStnorm_RANS=np.sqrt(np.trace(ReStReStT_RANS,axis1=1,axis2=2))
VeldPdx_RANS=np.einsum('ab,ab->a', Vel_RANS, GradP_RANS)
dPdx2Vel2_RANS=np.einsum('xy,xv->x', np.power(GradP_RANS,2), np.power(Vel_RANS,2))
Lturb_RANS=np.power(K,1.5)/(Eps+SMALL) #integral lenght scale
Vort_jGrad_ij_RANS=np.einsum('vu,vyu->vyu', VortVec, GradVel)
Vort_jGrad_ijVort_kGradik_RANS=np.sum(np.triu(np.einsum('vuy,vuw->vuyw', Vort_jGrad_ij_RANS, Vort_jGrad_ij_RANS),0),axis=(1,2,3))
U_jGrad_ij_RANS=np.einsum('vu,vyu->vyu', Vel_RANS, GradVel_RANS)
U_iU_jGrad_ij_RANS=np.einsum('vu,vuy->v', Vel_RANS, U_jGrad_ij_RANS)
U_jGrad_ijU_kGradik_RANS=np.triu(np.einsum('vuy,vuw->vuyw', U_jGrad_ij_RANS, U_jGrad_ij_RANS),0)
UnUnU_jGrad_ijU_kGradik_RANS=np.einsum('vu,vyxw->v', (np.power(Vel_RANS,2)), U_jGrad_ijU_kGradik_RANS)
ReSt_ijDij_RANS=np.sum(np.triu(np.einsum('vux,vux->vux', ReStress_RANS, D_RANS)),axis=(1,2))

In [20]:
gradP=np.array([1,1,1])
gradK=np.array([1,1,1])
#Tensor permutação de Levi-Civita
eijk = np.zeros((3, 3, 3))
eijk[0, 1, 2] = eijk[1, 2, 0] = eijk[2, 0, 1] = 1
eijk[0, 2, 1] = eijk[2, 1, 0] = eijk[1, 0, 2] = -1
#Definição dos tensores GradP,GradK,S e Omega
GradP=np.einsum('ijk,i->jk',eijk,gradP)
GradK=np.einsum('ijk,i->jk',eijk,gradK)
S=np.array([[2,1,-1],[1,1.5,-1.5],[-1,1.5,-3.5]])
Omega=np.array([[0,-1,1],[1,0,-1],[-1,1,0]])

In [21]:
# tr(S)
Inv1=S
inv1=np.trace(Inv1)
# tr(S*S)
Inv2=np.einsum('ij,jk->ik',S,S)
inv2=np.trace(Inv2)
# tr(S*S*S)
Inv3=np.einsum('ij,jk->ik',Inv2,S)
inv3=np.trace(Inv3)
# tr(GradP*GradP)
Inv4=np.einsum('ij,jk->ik',GradP,GradP)
inv4=np.trace(Inv4)
# tr(GradK*GradK)
Inv5=np.einsum('ij,jk->ik',GradK,GradK)
inv5=np.trace(Inv5)
# tr(Omega*Omega)
Inv6=np.einsum('ij,jk->ik',Omega,Omega)
inv6=np.trace(Inv6)
# tr(GradP*GradP*S)
Inv7=np.einsum('ij,jk->ik',Inv4,S)
inv7=np.trace(Inv7)
# tr(GradK*GradK*S)
Inv8=np.einsum('ij,jk->ik',Inv5,S)
inv8=np.trace(Inv8)
# tr(Omega*Omega*S)
Inv9=np.einsum('ij,jk->ik',Inv6,S)
inv9=np.trace(Inv9)
# tr(GradP*GradP*S*S)
Inv10=np.einsum('ij,jk->ik',Inv4,Inv2)
inv10=np.trace(Inv10)
# tr(GradK*GradK*S*S)
Inv11=np.einsum('ij,jk->ik',Inv5,Inv2)
inv11=np.trace(Inv11)
# tr(Omega*Omega*S*S)
Inv12=np.einsum('ij,jk->ik',Inv6,Inv2)
inv12=np.trace(Inv12)
# tr(GradP*GradK)
Inv13=np.einsum('ij,jk->ik',GradP,GradK)
inv13=np.trace(Inv13)
# tr(GradP*Omega)
Inv14=np.einsum('ij,jk->ik',GradP,Omega)
inv14=np.trace(Inv14)
# tr(GradK*Omega)
Inv15=np.einsum('ij,jk->ik',GradK,Omega)
inv15=np.trace(Inv15)
# tr(GradP*GradP*S*GradP*S*S)
Inv16=np.einsum('ij,jk->ik',Inv7,GradP)
Inv16=np.einsum('ij,jk->ik',Inv16,Inv2)
inv16=np.trace(Inv16)
# tr(GradK*GradK*S*GradK*S*S)
Inv17=np.einsum('ij,jk->ik',Inv8,GradK)
Inv17=np.einsum('ij,jk->ik',Inv17,Inv2)
inv17=np.trace(Inv17)
# tr(Omega*Omega*S*Omega*S*S)
Inv18=np.einsum('ij,jk->ik',Inv9,Omega)
Inv18=np.einsum('ij,jk->ik',Inv18,Inv2)
inv18=np.trace(Inv18)
# tr(GradP*GradK*S)
Inv19=np.einsum('ij,jk->ik',Inv13,S)
inv19=np.trace(Inv19)
# tr(GradP*Omega*S)
Inv20=np.einsum('ij,jk->ik',Inv14,S)
inv20=np.trace(Inv20)
# tr(GradK*Omega*S)
Inv21=np.einsum('ij,jk->ik',Inv15,S)
inv21=np.trace(Inv21)
# tr(GradP*GradK*S*S)
Inv22=np.einsum('ij,jk->ik',Inv13,Inv2)
inv22=np.trace(Inv22)
# tr(GradP*Omega*S*S)
Inv23=np.einsum('ij,jk->ik',Inv14,Inv2)
inv23=np.trace(Inv23)
# tr(GradK*Omega*S*S)
Inv24=np.einsum('ij,jk->ik',Inv15,Inv2)
inv24=np.trace(Inv24)
# tr(GradP*GradP*GradK*S)
Inv25=np.einsum('ij,jk->ik',GradP,Inv19)
inv25=np.trace(Inv25)
# tr(GradP*GradP*Omega*S)
Inv26=np.einsum('ij,jk->ik',GradP,Inv20)
inv26=np.trace(Inv26)
# tr(GradK*GradK*Omega*S)
Inv27=np.einsum('ij,jk->ik',GradK,Inv21)
inv27=np.trace(Inv27)
# tr(GradK*GradK*GradP*S)
Inv28=np.einsum('ij,jk->ik',Inv5,GradP)
Inv28=np.einsum('ij,jk->ik',Inv28,S)
inv28=np.trace(Inv28)
# tr(Omega*Omega*GradP*S)
Inv29=np.einsum('ij,jk->ik',Inv6,GradP)
Inv29=np.einsum('ij,jk->ik',Inv29,S)
inv29=np.trace(Inv29)
# tr(Omega*Omega*GradK*S)
Inv30=np.einsum('ij,jk->ik',Inv6,GradK)
Inv30=np.einsum('ij,jk->ik',Inv30,S)
inv30=np.trace(Inv30)
# tr(GradP*GradP*GradK*S*S)
Inv31=np.einsum('ij,jk->ik',GradP,Inv22)
inv31=np.trace(Inv31)
# tr(GradP*GradP*Omega*S*S)
Inv32=np.einsum('ij,jk->ik',GradP,Inv23)
inv32=np.trace(Inv32)
# tr(GradK*GradK*Omega*S*S)
Inv33=np.einsum('ij,jk->ik',GradK,Inv24)
inv33=np.trace(Inv33)
# tr(GradK*GradK*GradP*S*S)
Inv34=np.einsum('ij,jk->ik',Inv28,S)
inv34=np.trace(Inv34)
# tr(Omega*Omega*GradP*S*S)
Inv35=np.einsum('ij,jk->ik',Inv29,S)
inv35=np.trace(Inv35)
# tr(Omega*Omega*GradK*S*S)
Inv36=np.einsum('ij,jk->ik',Inv30,S)
inv36=np.trace(Inv36)
# tr(GradP*GradP*S*GradK*S*S)
Inv37=np.einsum('ij,jk->ik',Inv7,GradK)
Inv37=np.einsum('ij,jk->ik',Inv37,Inv2)
inv37=np.trace(Inv37)
# tr(GradK*GradK*S*GradP*S*S)
Inv38=np.einsum('ij,jk->ik',Inv8,GradP)
Inv38=np.einsum('ij,jk->ik',Inv38,Inv2)
inv38=np.trace(Inv38)
# tr(Omega*Omega*S*GradP*S*S)
Inv39=np.einsum('ij,jk->ik',Inv9,GradP)
Inv39=np.einsum('ij,jk->ik',Inv39,Inv2)
inv39=np.trace(Inv39)
# tr(GradP*GradP*S*Omega*S*S)
Inv40=np.einsum('ij,jk->ik',Inv7,Omega)
Inv40=np.einsum('ij,jk->ik',Inv40,Inv2)
inv40=np.trace(Inv40)
# tr(GradK*GradK*S*Omega*S*S)
Inv41=np.einsum('ij,jk->ik',Inv8,Omega)
Inv41=np.einsum('ij,jk->ik',Inv41,Inv2)
inv41=np.trace(Inv41)
# tr(Omega*Omega*S*GradK*S*S)
Inv42=np.einsum('ij,jk->ik',Inv9,GradK)
Inv42=np.einsum('ij,jk->ik',Inv39,Inv2)
inv42=np.trace(Inv39)
# tr(GradP*GradK*Omega)
Inv43=np.einsum('ij,jk->ik',GradP,GradK)
Inv43=np.einsum('ij,jk->ik',Inv43,Omega)
inv43=np.trace(Inv43)
# tr(GradP*GradK*Omega*S)
Inv44=np.einsum('ij,jk->ik',Inc43,S)
inv44=np.trace(Inv44)
# tr(GradP*GradK*Omega*S)
Inv45=np.einsum('ij,jk->ik',Inv14,GradK)
Inv45=np.einsum('ij,jk->ik',Inv45,S)
inv45=np.trace(Inv45)
# tr(GradP*GradK*Omega*S*S)
Inv46=np.einsum('ij,jk->ik',Inc43,Inv2)
inv46=np.trace(Inv46)
# tr(GradP*GradK*Omega*S*S)
Inv47=np.einsum('ij,jk->ik',Inv14,GradK)
Inv47=np.einsum('ij,jk->ik',Inv47,Inv2)
inv47=np.trace(Inv47)
# tr(GradP*GradK*S*Omega*S*S)
Inv48=np.einsum('ij,jk->ik',Inv14,S)
Inv48=np.einsum('ij,jk->ik',Inv48,Omega)
Inv48=np.einsum('ij,jk->ik',Inv48,Inv2)
inv48=np.trace(Inv48)

In [ ]:
# Non-dimensionalized Q criterion
inv49=((1/(pi))*np.arccos((np.power(Wnorm,2)-(np.power(Dnorm,2)))/(np.power(Wnorm,2)+(np.power(Dnorm,2)))))
# Turbulence intensity
kTurbNorm=K/(0.5*(np.einsum('ab,ab->a', Vel, Vel))+K+SMALL)
# Turbulence Reynolds number
LamVisc=0.001
Dens=1
dturb=1
ReTurb=np.minimum(((np.sqrt(K)*L_turb)/(50*LamVisc)),2.)
# Pressure gradient along streamline
PgradSL=VeldPdx/(np.sqrt(dPdx2Vel2)+np.abs(VeldPdx)+SMALL)
# Ratio of turbulent time scale to mean strain time scale
TurbTimeScale_StrainTimeScale=(Dnorm*K)/(Dnorm*K+Eps+SMALL)
# Viscosity ratio
ViscRatio=TurbVisc/(100*LamVisc+TurbVisc+SMALL)
# Ratio of pressure normal stresses to normal shear stresses
PressNormalStress_NormalShearStress=(np.sqrt(np.einsum('ab,ab->a', GradP, GradP)))/\
(np.sqrt(np.einsum('ab,ab->a', GradP, GradP))+0.5*Dens*(np.einsum('ab,ab->a', DivVel, DivVel))+SMALL)
# Vortex stretching
VortexStretch=np.sqrt(Vort_jGrad_ijVort_kGradik)/(np.sqrt(Vort_jGrad_ijVort_kGradik)+Dnorm+SMALL)
#Marker of Gorle et al deviation from parallel shear flow
GorlePar=np.abs(U_iU_jGrad_ij)/(np.sqrt(UnUnU_jGrad_ijU_kGradik)+np.abs(U_iU_jGrad_ij)+SMALL)
#Ratio of convection to production of k
#Ratio of total Reynolds stresses to normal Reynolds stresses
ReSt_Total_Normal=ReStnorm/(K+ReStnorm)
#Cubic eddy viscosity comparison